# fine-Tuning (LoRA)

In [ ]:
# !pip install transformers datasets peft trl accelerate bitsandbytes
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [ ]:
token = "your_token"

# ✅ 데이터셋 로드 (train, validation)
dataset = load_dataset("json", data_files={
    "train": "preprocessing/new/train/gap-dev_one_sft.jsonl",
    "validation": "./preprocessing/gap/gap-validation_sft.jsonl"
})

# ✅ 모델 로드 (8bit + padding 토큰 설정)
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token=token)
tokenizer.pad_token = tokenizer.eos_token  # padding 토큰을 eos로 지정

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True,
    token=token
)

lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, task_type="CAUSAL_LM")
model = get_peft_model(model, lora_config)

# ✅ TrainingArguments (메모리 최적화)
training_args = TrainingArguments(
    output_dir="./finetuned/llama-lora-3.2-3B_e10-gap-dev_one",
    per_device_train_batch_size=8,  # ✅ 메모리 맞춰서
    gradient_accumulation_steps=2,  # ✅ batch 총합 8
    learning_rate=2e-5,
    num_train_epochs=10,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=True,
    optim="paged_adamw_8bit",
    warmup_steps=50,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    report_to="none"
)


In [ ]:
# ✅ SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer
)

# ✅ 학습 시작
trainer.train()

# ✅ 저장
trainer.model.save_pretrained("./finetuned/llama-lora-3.2-3B_e10-gap-dev_one")

# inference

In [ ]:
import os
import json
import time
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig
import re

In [ ]:
# ✅ 토큰 및 모델/어댑터 경로 설정
token = "your_token"
base_model_name = "meta-llama/Llama-3.2-3B-Instruct"  # Base 모델 이름
# ✅ 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_name, token=token, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # padding 토큰 설정

# ✅ LoRA 어댑터 로드 및 모델 결합
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    trust_remote_code=True,
    token=token
)

Adpter적용시

In [ ]:
# adapter_path = "./finetuned/llama/llama-lora-3.2-3B_e10-gap-dev_npe"  # LoRA 어댑터 저장 경로
# model = PeftModel.from_pretrained(model, adapter_path, token=token)  # LoRA 적용

In [ ]:
# ✅ Pipeline 구성
llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=20,  # 생성할 최대 토큰 수 (필요 시 조절)
    do_sample=False  # 랜덤성 제거 (일관된 답변)
)

# ✅ 추론 함수
def query_llama(prompt):
    response = llama_pipeline(prompt)[0]["generated_text"]
    return response.replace(prompt, "").strip()

# ✅ 정답 클리닝 함수
def clean_answer(raw_answer):
    return re.sub(r'[^a-zA-Z]', '', raw_answer).strip().upper()

In [ ]:
name_list = ["Distance_factor_NPE_sub-corpus"]  # 사용할 데이터셋 이름

# ✅ 메인 실행
for name in name_list:
    json_file_path = os.path.join(os.getcwd(), "preprocessing", "test","npe", f"{name}.json")
    csv_file_path = os.path.join(os.getcwd(), "output", "zero", "LLaMA-zero", f"{name}.csv")  # 저장 파일명 수정

    # JSON 데이터 로드
    with open(json_file_path, "r", encoding="utf-8") as json_file:
        test_data = json.load(json_file)

    # 기존 CSV 확인 및 이미 처리된 text_id 로드
    if os.path.exists(csv_file_path):
        df_existing = pd.read_csv(csv_file_path, encoding="utf-8")
        processed_ids = set(df_existing["text_id"].tolist())
        print(f"🔄 기존 데이터 {len(processed_ids)}개 로드 완료. 이어서 실행합니다.")
    else:
        df_existing = pd.DataFrame()
        processed_ids = set()

    file_exists = os.path.exists(csv_file_path)  # 헤더 포함 여부 결정

    # ✅ 데이터 처리
    for data in test_data:
        if data["text_id"] in processed_ids:  # 이미 처리된 경우 스킵
            continue

        # 프롬프트 생성
        prompt = f'''Question: In the sentence "{data["text"]}", what does "{data["target"]}" refer to?
Options:
(A) {data["options"]["A"]}
(B) {data["options"]["B"]}

Answer only with "A" if (A) is correct, "B" if (B) is correct, or "Neither" if none of them are correct. Do not provide explanations.
Answer:'''

#         prompt = f'''Question: In the sentence "{data["text"]}", what should replace "{data["target"]}"?
# Options:
# (A) {data["options"]["A"]}
# (B) {data["options"]["B"]}
# Answer only with "A" if (A) is correct, "B" if (B) is correct, or "Neither" if none of them are correct. Do not provide explanations.
# Answer:'''

        max_retries = 10
        retry_count = 0

        # ✅ 응답 획득 루프
        while retry_count < max_retries:
            llama_response = query_llama(prompt)

            # 빈 응답 확인
            if not llama_response.strip():
                print(f"⚠️ [경고] 빈 응답. text_id: {data['text_id']}, 20초 후 재시도 ({retry_count + 1}/{max_retries})")
                time.sleep(20)
                retry_count += 1
                continue

            # 첫 단어 정제 및 확인
            llama_answer = clean_answer(llama_response.split()[0])

            # 정상 응답인지 확인
            if llama_answer in ["A", "B", "NEITHER"]:
                break
            else:
                print(f"⚠️ [경고] 이상한 응답 '{llama_answer}'. text_id: {data['text_id']}, 20초 후 재시도 ({retry_count + 1}/{max_retries})")
                time.sleep(20)
                retry_count += 1

        # ✅ 최대 재시도 실패 시
        if retry_count == max_retries:
            print(f"❌ [에러] 최대 재시도 초과. text_id: {data['text_id']}, 'API FAILED'로 저장")
            llama_answer = "API FAILED"
            correct = False
        else:
            # 정답 비교
            correct = (llama_answer == data["answer"].strip().upper())

        # 결과 저장
        result = {
            "text_id": data["text_id"],
            "text": data["text"],
            "target": data["target"],
            "expected_answer": data["answer"].strip().upper(),
            "llama_answer": llama_answer,
            "correct": correct
        }

        # ✅ CSV로 바로 저장
        df_temp = pd.DataFrame([result])
        df_temp.to_csv(csv_file_path, mode="a", index=False, header=not file_exists, encoding="utf-8")
        file_exists = True  # 첫 저장 이후부터 header=False 유지

    print(f"✅ [{name}] 모든 데이터 처리 완료: {csv_file_path}")